# Import

In [9]:
%pip install cykhash
%pip install pyrosm==0.6.2
%pip install pyrosm
%pip install pyrosm_proto

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement pyrosm_proto (from versions: none)
ERROR: No matching distribution found for pyrosm_proto

[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [10]:
import pickle
import networkx as nx
import geopandas as gpd
from tqdm import tqdm

import rasterio
import rasterio.mask
from rasterio.enums import Resampling
print(f'rasterio version: {rasterio.__version__}')

import os
import matplotlib.pyplot as plt

from shapely.geometry import Point
import pandas as pd
import zipfile
import shutil

import torch
import numpy as np

import pandas as pd
from pyrosm import OSM, get_data
from geopy.distance import geodesic

from sklearn.neighbors import BallTree
import numpy as np

rasterio version: 1.3.10


ModuleNotFoundError: No module named 'pyrosm_proto'

# Functions

In [ ]:
def get_geometry_df(path2graph):
    """
    Extract geometry information from a graph and return as a GeoDataFrame.
    
    Args:
    path2graph (str): Path to the graph file (pickle format).

    Returns:
    GeoDataFrame: Contains zone_id and geometry of nodes with EPSG:4326 CRS.
    """
    with open(path2graph, 'rb') as f:
        G = pickle.load(f)

    geometry = nx.get_node_attributes(G, 'geometry')
    geometry_df = gpd.GeoDataFrame(data={'zone_id': geometry.keys(), 'geometry': geometry.values()},
                                   geometry='geometry', crs='epsg:4326')
    
    return geometry_df


def get_imgs_lst(geometry_df, dataset):
    """
    Generate a list of image arrays by masking the dataset with geometries.

    Args:
    geometry_df (GeoDataFrame): DataFrame containing geometries.
    dataset (rasterio.DatasetReader): Raster dataset to mask.

    Returns:
    list: List of image arrays corresponding to the geometries.
    """
    c, e = 0, 1
    imgs_lst = []

    for i in tqdm(geometry_df.values, desc='Get images'):
        poly = gpd.GeoDataFrame(data=[0], geometry=[i[1]], crs='epsg:4326')

        try:
            out_image, out_transform = rasterio.mask.mask(dataset,
                                                          [poly.geometry.values[0]],
                                                          crop=True)
            imgs_lst.append(out_image.tolist())
            c += 1
        except Exception as er:
            print(f'Не удалось обработать изображение №{e}')
            print(er)
            e += 1
    return imgs_lst


def resize_array(arr, target_shape):
    """
    Resize a 3D numpy array to a target shape, padding with zeros if necessary.

    Args:
    arr (numpy.ndarray): Input array to be resized.
    target_shape (tuple): Target shape (depth, height, width).

    Returns:
    numpy.ndarray: Resized array with the target shape.
    """
    # Resize or pad depth (axis 0)
    if arr.shape[0] > target_shape[0]:
        arr = arr[:target_shape[0], :, :]
    else:
        pad_width = [(0, target_shape[0] - arr.shape[0]), (0, 0), (0, 0)]
        arr = np.pad(arr, pad_width, mode='constant', constant_values=0)

    # Resize or pad height (axis 1)
    if arr.shape[1] > target_shape[1]:
        arr = arr[:, :target_shape[1], :]
    else:
        pad_width = [(0, 0), (0, target_shape[1] - arr.shape[1]), (0, 0)]
        arr = np.pad(arr, pad_width, mode='constant', constant_values=0)

    # Resize or pad width (axis 2)
    if arr.shape[2] > target_shape[2]:
        arr = arr[:, :, :target_shape[2]]
    else:
        pad_width = [(0, 0), (0, 0), (0, target_shape[2] - arr.shape[2])]
        arr = np.pad(arr, pad_width, mode='constant', constant_values=0)

    return arr


# Placing Images in a Graph

In [4]:
# List of city codes and corresponding dictionary
city_codes = ['287', '812', '777', '473']
code2name = {'777': 'Moscow', '812': 'Saint-Petersburg', '473': 'Sochi', '287': 'Kazan'}

# Paths to required folders
path2graphs = '/home/jupyter/datasphere/s3/s3-sirius/sirius_2024_participants/data/graphs'
path2tiffs = '/home/jupyter/datasphere/s3/s3-sirius/sirius_2024_participants/data/raster_data'
path2datasets = '../../../data/graph_preprocessing'
path2save = '../../data/graph_with_images'

In [5]:
# If the images are in the zip, create and unzip them

if not os.path.exists(path2save):
    os.mkdir(path2save)


if tiff_in_zip := False:
    path2dir = f'{path2tiffs}/raster_data'
    os.mkdir(path2dir)
    for code in tqdm(city_codes):
        pathzip = f'{path2tiffs}/city_{code}.tiff.zip'

        with zipfile.ZipFile(pathzip) as zip_file:
            for member in zip_file.namelist():
                filename = os.path.basename(member)
                if not filename:
                    continue
                source = zip_file.open(member)
                target = open(os.path.join(path2dir, filename), "wb")
                with source, target:
                    shutil.copyfileobj(source, target)

In [ ]:
for code in city_codes:
    print(f'The processing of {code2name[code]} begins')
    
    path2graph = f'{path2graphs}/graph_{code}_buffer_5_road_connected.gpickle'
    path2tiff = f'{path2tiffs}/raster_data/city_{code}.tiff'
    
    geometry_df = get_geometry_df(path2graph)
    raster_dataset = rasterio.open(path2tiff)
    dataset = torch.load(f'{path2dataset}/graph_with_cv_full_{code}.pickle')
    
    imgs_lst = get_imgs_lst(geometry_df, raster_dataset)
    imgs_lst_correct = [resize_array(np.array(i), (12, 52, 52)) for i in tqdm(imgs_lst, desc='Image resizing')]
    dataset.imgs = torch.tensor(imgs_lst_correct)

    torch.save(dataset, f'{path2save}/graph_with_cv_full_and_images_{code}.pickle')
    print(f'The new {code2name[code]} graph has been preserved')